In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.dqn import DQN
from networks.lstm_dueling import LSTMDueling

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch

In [2]:
norm_cols = ['Volume XRP']
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 20
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/vinay/code/git/smart-broker/notebooks/../environment.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['rolling_price'] = self.df[self.price_typ].rolling(self.roll_period).sum()
/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [3]:
batch_size = 100
target_net = LSTMDueling(
    input_dim=batch_dur*3+3,
    output_dim=3,
)

policy_net = LSTMDueling(
    input_dim=batch_dur*3+3,
    output_dim=3,
)

dqn = DQN(
    env=env,
    env_type='vector',
    n_actions=3,
    log_freq=10,
    train_freq=3,
    batch_size=batch_size,
    w_sync_freq=1,
    memory_size=500,
    gamma=0.9995,
    step_size=0.01,
    episodes=500,
    target_net=target_net,
    policy_net=policy_net,
    loss_func=nn.SmoothL1Loss(),
    optimizer=torch.optim.Adam(policy_net.parameters(), lr=0.00008),
    load_pretrained=False,
    save_pretrained=False,
    model_path='../models/dqn_lstm',
    network_type='lstm',
)

dqn.run(5000)

collecting experience...
0..
Ep: 10 | L: 3847.866 | R: 0.52 | R.Avg.R: 0.51 | P: 0.03 | R.Avg P: 0.02 | B: 91.81 | R.Avg B: 89.2 | R.N_Units: 46
Ep: 20 | L: 18969.079 | R: 0.51 | R.Avg.R: 0.52 | P: -1.18 | R.Avg P: 0.39 | B: 86.63 | R.Avg B: 89.54 | R.N_Units: 46
Ep: 30 | L: 17981.421 | R: 0.51 | R.Avg.R: 0.52 | P: -0.38 | R.Avg P: 0.36 | B: 85.49 | R.Avg B: 89.14 | R.N_Units: 48
Ep: 40 | L: 7579.437 | R: 0.53 | R.Avg.R: 0.52 | P: 1.82 | R.Avg P: 0.55 | B: 91.96 | R.Avg B: 89.17 | R.N_Units: 48
Ep: 50 | L: 377.629 | R: 0.54 | R.Avg.R: 0.53 | P: 3.5 | R.Avg P: 1.49 | B: 46.06 | R.Avg B: 79.67 | R.N_Units: 94
Ep: 60 | L: 424.434 | R: 0.54 | R.Avg.R: 0.54 | P: 3.78 | R.Avg P: 2.44 | B: 52.86 | R.Avg B: 57.29 | R.N_Units: 195
Ep: 70 | L: 386.044 | R: 0.53 | R.Avg.R: 0.53 | P: 2.87 | R.Avg P: 2.66 | B: 61.12 | R.Avg B: 49.83 | R.N_Units: 228
Ep: 80 | L: 374.904 | R: 0.52 | R.Avg.R: 0.54 | P: 0.78 | R.Avg P: 2.97 | B: 46.02 | R.Avg B: 53.07 | R.N_Units: 216
Ep: 90 | L: 409.161 | R: 0.55 | R.

KeyboardInterrupt: 

### Visualizations

### Traning

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in dqn.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')